In [ ]:
# Allows you to use modified modules without rebooting the kernel
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

from statsmodels.tsa.api import VAR
from plotly.express import line

I will use VAR method for forecasting time series. In future LSTM usage is possible.

General ideas. 
Me can observe $$a_k, b_k, p_k => a(t_k), b(t_k)$$.
Moreover we can calculate two more statistical series. Where one of them is one dimensional, 
but other is M-dimensional.

# 1. Load up data

In [ ]:
magData = pd.read_csv("../src/datasets/2020_ydhm_id.csv")
magData.dropna(inplace=True)

N = 36_000  # units in sec so it's 10 hours
# Get subset of original dataset
dataset = magData.iloc[:N]
dataset.index = pd.DatetimeIndex(dataset["ydhm_id"])
dataset = dataset.drop(columns=["ydhm_id"])
# Free space
del magData

dataset.diff().head()

# 2. Apply VAR model on Bx series with different replenishment.

First of all we need to calculate new series. Therefore run cells below.

In [ ]:
dataset.diff()

In [ ]:
from magfield.em.mixture import DynamicMixture
from tensorflow_probability import distributions

TIME = dataset.index.values
WINDOW = (4200, 1)

mix_dBX = DynamicMixture(
    num_comps=3,
    distrib=distributions.Normal,
    time_span=TIME,
    window_shape=WINDOW,
)

dBx = dataset.diff()["Bx"].dropna().values
mix_dBX.predict_light(data=dBx)

# Get rid of unaccounted (useless) counts
dataset = dataset.iloc[WINDOW[0] :]

In [ ]:
mix_dBX.show_parameters()

In [ ]:
a, b = mix_dBX.reconstruct_process_coef()
line(a)

In [ ]:
print(len(a), len(dataset["Bx"]), len(mix_dBX.parameters["probs"]))

In [ ]:
dataset["a_k"] = mix_dBX.parameters["mus"]
dataset["b_k"] = mix_dBX.parameters["sigmas"]
dataset["p_k"] = mix_dBX.parameters["probs"]

dataset["a_t"] = a
dataset["b_t"] = b
dataset.head()

In [ ]:
dataset.head()

In [ ]:
# make a VAR model
model = VAR(dataset)

In [ ]:
resultVAR = model.fit(maxlags=2)
resultVAR.summary()
# resultVAR.plot()  # Optionally plot all dataset series